In [1]:
#view all dataframe
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)

library(corrplot)
library("Hmisc")

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: 'Hmisc'

The following objects are masked from 'package:base':

    format.pval, units



---
### data import

In [2]:
clinical1 <- read.csv("data.csv", header = TRUE)
head(clinical1)

bcr.patient.barcode,yearstobirth,vitalstatus,daystodeath,daystolastfollowup,tumortissuesite,pathologicstage,pathologyTstage,pathologyNstage,pathologyMstage,gender,histologicaltype,extrathyroidalextension,residualtumor,numberoflymphnodes,multifocality,tumorsize,race,ethnicity,name,days,age.group,BRAF
TCGA.BJ.A0YZ,65,0,NA,754,thyroid,stage ii,t2,n0,m0,male,thyroid papillary carcinoma . classical/usual,none,r0,NA,unifocal,NA,white,not hispanic or latino,TCGA.BJ.A0YZ.01A,754,60.69,0
TCGA.BJ.A0Z0,55,0,NA,419,thyroid,stage ii,t2,n0,mx,male,thyroid papillary carcinoma . classical/usual,none,r0,0,unifocal,NA,NA,not hispanic or latino,TCGA.BJ.A0Z0.01A,419,50.59,0
TCGA.BJ.A0Z2,57,0,NA,1514,thyroid,stage ivc,t2,n0,m1,male,thyroid papillary carcinoma . follicular (>= 99% follicular patterned),none,rx,NA,unifocal,2.8,black or african american,not hispanic or latino,TCGA.BJ.A0Z2.01A,1514,50.59,0
TCGA.BJ.A0Z3,33,0,NA,497,thyroid,stage i,t1,n0,m0,female,thyroid papillary carcinoma . classical/usual,none,rx,0,multifocal,1.3,white,not hispanic or latino,TCGA.BJ.A0Z3.01A,497,30.39,1
TCGA.BJ.A0Z5,58,0,NA,448,thyroid,stage iii,t3,n1a,m0,male,thyroid papillary carcinoma . classical/usual,NA,NA,4,NA,NA,white,not hispanic or latino,TCGA.BJ.A0Z5.01A,448,50.59,0
TCGA.BJ.A0Z9,57,0,NA,1257,thyroid,stage iii,t3,n0,mx,female,thyroid papillary carcinoma . tall cell (>= 50% tall cell features),minimal (t3),r1,0,multifocal,2.8,white,not hispanic or latino,TCGA.BJ.A0Z9.01A,1257,50.59,1


In [3]:
clinical2 <- clinical1[, c("multifocality", "BRAF", "name")]
clinical2$name <- substr(clinical1$name, 1, 15)

In [4]:
# converting focality

clinical2$focality <- as.character(clinical1$multifocality)
clinical2$focality[clinical1$multifocality == "multifocal"] <- 1
clinical2$focality[clinical1$multifocality == "unifocal"] <- 0
clinical2$focality <- as.numeric(clinical2$focality)

In [9]:
#separate BRAF_p/focality

clinical_p <- clinical2[clinical2$BRAF %in% "1",]

clinical_p_uni <- clinical_p[clinical_p$focality %in% "0",]
head(clinical_p_uni)

clinical_p_multi <- clinical_p[clinical_p$focality %in% "1",]
head(clinical_p_multi)

,multifocality,BRAF,name,focality
8,unifocal,1,TCGA.BJ.A0ZB.01,0
19,unifocal,1,TCGA.BJ.A28R.01,0
26,unifocal,1,TCGA.BJ.A290.01,0
35,unifocal,1,TCGA.BJ.A3PR.01,0
37,unifocal,1,TCGA.BJ.A3PU.01,0
48,unifocal,1,TCGA.BJ.A4O9.01,0


,multifocality,BRAF,name,focality
4,multifocal,1,TCGA.BJ.A0Z3.01,1
6,multifocal,1,TCGA.BJ.A0Z9.01,1
15,multifocal,1,TCGA.BJ.A18Z.01,1
20,multifocal,1,TCGA.BJ.A28S.01,1
24,multifocal,1,TCGA.BJ.A28X.01,1
31,multifocal,1,TCGA.BJ.A2NA.01,1


---
### loading mRNA

In [10]:
load("mRNA.RData")

In [11]:
#miRNA df name col 분리
name <- rownames(mRNA)
rownames(mRNA) <- NULL
mRNA <- cbind(name,mRNA)

In [12]:
#mRNA name 형식 변경
#TCGA-DJ-A2Q5-01A-11R-A18B-13 type >>> TCGA.BJ.A0YZ.01 type(clinical2)
mRNA$name <- gsub("-", ".", mRNA$name)
mRNA$name <- substr(mRNA$name, 1, 15)

In [13]:
head(mRNA)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt'의 길이가 최대 포맷길이 8192를 초과합니다

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_latex.data.frame(obj)
9. repr_matrix_generic(obj, sprintf("\\begin{tabular}{%s}\n%%s%%s\\end{tabular}\n", 
 .     cols), "%s\\\\\n\\hline\n", "  &", " %s &", "%s", "\t%s\\\\\n", 
 .     "%s &", " %s &", escape_fun = latex_escape_vec, ...)
10. sprintf(wrap, header, bod

name,A1BG,A2BP1,A2LD1,A2M,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,AAK1,AAMP,AANAT,AARS2,AARSD1,AARS,AASDHPPT,AASDH,AASS,AATF,AATK,ABAT,ABCA10,ABCA11P,ABCA12,ABCA13,ABCA17P,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB10,ABCB1,ABCB4,ABCB6,ABCB7,ABCB8,ABCB9,ABCC10,ABCC11,ABCC1,ABCC2,ABCC3,ABCC4,ABCC5,ABCC6P1,ABCC6P2,ABCC6,ABCC8,ABCC9,ABCD1,ABCD2,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABCG2,ABCG4,ABCG5,ABHD10,ABHD11,ABHD12B,ABHD12,ABHD13,ABHD14A,ABHD14B,ABHD15,ABHD1,ABHD2,ABHD3,ABHD4,ABHD5,ABHD6,ABHD8,ABI1,ABI2,ABI3BP,ABI3,ABL1,ABL2,ABLIM1,ABLIM2,ABLIM3,ABO,ABP1,ABR,ABT1,ABTB1,ABTB2,ACAA1,ACAA2,...,ZNF791,ZNF792,ZNF793,ZNF799,ZNF79,ZNF7,ZNF800,ZNF804A,ZNF805,ZNF808,ZNF80,ZNF813,ZNF814,ZNF815,ZNF816A,ZNF81,ZNF821,ZNF823,ZNF826,ZNF827,ZNF828,ZNF829,ZNF830,ZNF831,ZNF833,ZNF835,ZNF836,ZNF837,ZNF839,ZNF83,ZNF841,ZNF843,ZNF844,ZNF845,ZNF846,ZNF84,ZNF853,ZNF85,ZNF860,ZNF862,ZNF876P,ZNF878,ZNF879,ZNF880,ZNF883,ZNF8,ZNF90,ZNF91,ZNF92,ZNF93,ZNF98,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZNRF3,ZP1,ZP3,ZPLD1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN12P1,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN1,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN29,ZSCAN2,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSWIM1,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22
TCGA.4C.A93U.01,108.7681,0.0000,80.1401,13369.058,256.0386,1.9324,665.7005,371.0145,144.9275,736.7150,842.5121,2551.208,0.4831,483.9903,571.4976,3375.362,605.7971,184.0580,288.4058,1251.6908,50.7246,1688.406,3.3816,117.6763,27.5362,417.3913,12.0773,641.0628,1648.309,3061.836,29.9517,198.5507,13.0435,901.4493,19.8068,6.2802,405.3140,40.5797,11.5942,129.0580,487.4396,688.8889,129.4734,500.4831,0.0000,1368.599,0.4831,3239.6135,72.4638,185.5072,68.1159,86.9565,201.9324,0.0000,200.4831,257.0048,13.5266,1271.981,779.2271,1054.5894,1430.435,1170.5314,946.8599,674.8792,128.5024,2.8986,0.4831,772.9469,1012.0773,4.8309,2882.126,478.7440,731.8841,1998.068,261.3527,1.9324,1901.449,526.0870,1300.9662,498.0676,111.5942,545.8937,1197.0531,2698.551,1850.7246,232.8502,1544.928,1018.3575,2293.720,169.0821,82.1256,67.6329,6.7633,2303.8647,436.2319,616.4251,484.0580,1277.778,1811.1111,...,236.2319,116.9082,77.2947,58.9372,29.9517,422.3575,260.3865,4.3478,213.5266,181.1594,5.7198,40.0966,169.8647,16.4251,82.0966,71.0145,183.4396,34.7826,20.7729,247.8261,323.6715,8.2126,289.3720,33.8164,0.9662,21.2560,84.0580,57.9710,364.2512,753.6232,198.5507,5.7971,151.6908,81.6425,62.8019,494.2029,264.2512,73.9130,136.7150,259.4203,27.5362,2.8986,40.0966,98.5507,14.9758,116.0193,157.4879,285.0242,76.8116,31.4010,0.4831,2190.821,1131.401,197.1014,428.6957,493.2367,375.5845,731.8841,1414.9758,417.3913,0.9662,34.9420,0.4831,749.2754,1241.8019,49.2754,355.5556,12.4734,68.1159,95.6522,1825.121,0.9662,47.3430,83.1159,52.6570,40.5797,268.1159,134.2995,87.4396,28.9855,0.9662,149.2754,34.2995,444.9275,329.4686,240.0966,289.1401,112.5604,287.4396,144.5604,194.6860,67.6329,253.1401,865.7005,26.0870,1799.0338,4436.2319,869.0821,655.5556,48.3092
TCGA.BJ.A0YZ.01,251.7201,23.7401,64.3773,20192.378,286.1308,5.8309,622.2407,949.1878,112.8696,1657.6426,2230.7372,3315.285,2.0825,407.3303,867.5552,5159.100,748.0217,211.9950,294.0441,1120.7830,57.8925,3378.592,1.6660,110.4581,3.3319,3.3319,10.4123,965.4144,2717.618,2174.094,127.8634,241.5660,18.7422,481.0496,74.9688,4.1649,248.6464,209.0796,33.3195,512.4906,446.0641,726.7805,72.2282,409.4127,6.6639,2091.212,0.8330,1430.2374,117.4511,179.9250,0.0000,44.1483,47.0637,2.4990,692.2116,329.0296,2.0825,2283.632,1020.4082,900.8746,1491.462,1114.5356,1078.7172,303.2070,760.1000,64.1399,0.4165,940.0250,1123.2820,9.9958,4539.775,330.2791,2467.1179,5574.761,1105.3728,43.7318,1610.579,528.1133,2873.3861,1429.8209,862.9738,503.9567,900.2624,2020.825,23.7401,519.3669,2488.963,551.4369,6556.018,136.6097,222.8238,600.1666,0.8330,1192.8363,469.3878,362.3490,30.4040,10713.869,4476.4681,...,249.8959,106.2057,130.7788,98.2924,183.2570,278.8338,298.6256,2.9155,243.2320,311.5369,0.0000,78.71

In [14]:
selected_mRNA = read.csv("mRNA_selected.csv")

In [15]:
mRNA_f <- dplyr::bind_rows(mRNA, selected_mRNA)

Warning message in bind_rows_(x, .id):
"binding character and factor vector, coercing into character vector"

In [16]:
tail(mRNA_f)

ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt'의 길이가 최대 포맷길이 8192를 초과합니다

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_latex.data.frame(obj)
9. repr_matrix_generic(obj, sprintf("\\begin{tabular}{%s}\n%%s%%s\\end{tabular}\n", 
 .     cols), "%s\\\\\n\\hline\n", "  &", " %s &", "%s", "\t%s\\\\\n", 
 .     "%s &", " %s &", escape_fun = latex_escape_vec, ...)
10. sprintf(wrap, header, bod

,name,A1BG,A2BP1,A2LD1,A2M,A4GALT,A4GNT,AAAS,AACS,AADAT,AAGAB,AAK1,AAMP,AANAT,AARS2,AARSD1,AARS,AASDHPPT,AASDH,AASS,AATF,AATK,ABAT,ABCA10,ABCA11P,ABCA12,ABCA13,ABCA17P,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,ABCA7,ABCA8,ABCA9,ABCB10,ABCB1,ABCB4,ABCB6,ABCB7,ABCB8,ABCB9,ABCC10,ABCC11,ABCC1,ABCC2,ABCC3,ABCC4,ABCC5,ABCC6P1,ABCC6P2,ABCC6,ABCC8,ABCC9,ABCD1,ABCD2,ABCD3,ABCD4,ABCE1,ABCF1,ABCF2,ABCF3,ABCG1,ABCG2,ABCG4,ABCG5,ABHD10,ABHD11,ABHD12B,ABHD12,ABHD13,ABHD14A,ABHD14B,ABHD15,ABHD1,ABHD2,ABHD3,ABHD4,ABHD5,ABHD6,ABHD8,ABI1,ABI2,ABI3BP,ABI3,ABL1,ABL2,ABLIM1,ABLIM2,ABLIM3,ABO,ABP1,ABR,ABT1,ABTB1,ABTB2,ACAA1,ACAA2,...,ZNF815,ZNF816A,ZNF81,ZNF821,ZNF823,ZNF826,ZNF827,ZNF828,ZNF829,ZNF830,ZNF831,ZNF833,ZNF835,ZNF836,ZNF837,ZNF839,ZNF83,ZNF841,ZNF843,ZNF844,ZNF845,ZNF846,ZNF84,ZNF853,ZNF85,ZNF860,ZNF862,ZNF876P,ZNF878,ZNF879,ZNF880,ZNF883,ZNF8,ZNF90,ZNF91,ZNF92,ZNF93,ZNF98,ZNFX1,ZNHIT1,ZNHIT2,ZNHIT3,ZNHIT6,ZNRD1,ZNRF1,ZNRF2,ZNRF3,ZP1,ZP3,ZPLD1,ZRANB1,ZRANB2,ZRANB3,ZRSR2,ZSCAN12P1,ZSCAN12,ZSCAN16,ZSCAN18,ZSCAN1,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN29,ZSCAN2,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSWIM1,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZUFSP,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,ADGRL2,ADGRL4,C10ORF76,C15ORF56,C1ORF95,C6ORF89,CEP126,GALNT16,PDIA3P1,PRR29,RMDN2,TMEM242,TMEM248
497,TCGA.MK.A4N6.01,70.8471,0.7666,96.7574,7901.100,547.7194,0.0000,446.5312,502.1081,78.1909,899.5784,1136.068,2355.692,0.0000,431.5332,410.8854,2704.868,727.4818,257.9532,190.1112,942.8900,61.7095,1670.372,2.2997,70.2645,17.6313,54.8103,3.0663,1296.6194,2723.266,2522.422,78.9575,292.4492,11.1154,504.4078,66.3089,3.8329,306.6309,32.9628,7.6658,258.7160,436.1824,642.3917,142.6984,486.7765,3.4496,1527.022,2.2997,3004.599,230.3565,175.9295,37.9456,61.7095,150.6324,0.3833,95.8222,292.0659,9.9655,1413.568,616.7114,924.4921,1398.6202,1244.9214,1085.0901,730.9314,51.3607,56.7267,0.0000,495.2089,1112.3036,1.1499,3909.927,446.9145,394.0667,1807.206,435.0326,4.2162,2281.334,1049.4442,1097.3553,845.9180,134.5343,409.7355,1947.4665,2431.583,50.9774,130.3181,4886.930,899.5784,8213.109,306.2476,211.1920,111.5370,47.1445,3118.053,415.4849,389.8045,2407.8191,1727.099,2251.821,...,18.7811,110.7512,83.5569,228.0567,84.7068,41.0119,267.5355,344.5765,11.4987,308.5473,32.5795,6.1326,6.1326,127.6351,37.5623,240.7053,832.8862,227.6734,10.7321,443.8482,128.7850,79.7240,587.5814,555.0019,86.2399,78.9575,352.6255,32.9628,1.5332,54.4270,269.0686,18.0146,95.9371,119.5860,301.2649,120.7359,74.7413,14.9483,2955.922,869.6819,161.1690,387.9801,376.7727,229.6167,1040.245,1205.4427,1101.9548,0.7666,24.9138,7.2825,1162.8977,1281.966,61.7095,230.3565,26.7689,205.0594,79.3407,2414.335,2.6830,47.1445,144.8256,80.1073,54.0437,261.0195,193.5608,1.5332,22.6140,0.3833,158.2982,59.7930,307.0142,377.1560,274.0514,266.9030,123.8022,301.2649,189.2641,184.3618,36.4124,371.7900,1043.312,62.8593,2811.422,4632.426,1179.762,718.6662,411.6520,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,TCGA.MK.A4N7.01,46.7046,1018.9109,132.6771,8726.942,174.3753,0.9114,671.0716,321.7134,232.0954,933.2422,1530.797,2793.347,0.6076,329.6058,615.1743,2436.394,1112.7820,205.9695,401.0025,1151.3633,190.4762,1559.657,5.1644,153.1070,1.5189,201.7164,29.4676,2540.8855,7213.792,1285.031,101.1620,301.3595,49.8215,426.5208,551.0747,11.2402,259.1327,33.4169,4.5568,202.9863,484.2409,620.6425,155.3915,497.0001,13.0630,1670.540,3.3417,3172.780,173.1602,132.7561,2.1265,64.0996,170.1223,4.5568,203.2354,315.6376,1.8227,1619.807,998.8608,966.6591,1036.2269,1182.0460,1064.1756,913.7997,264.9047,8.8099,0.6076,918.3565,1391.3572,1.2152,2804.587,457.2036,853.7009,2118.934,274.9297,15.1895,3053.695,525.8601,1269.2337,557.7580,170.1223,411.0276,1462.3741,3650.642,45.2647,88.4028,4657.097,565.3528,7827.447,452.0392,81.1119,186.2231,6.0758,1657.477,483.3295,487.5826,1251.9177,1807.549,3706.539,...,32.2017,120.9509,85.3649,375.3870,62.5807,13.9743,171.9450,473.9121,17.9236,389.7623,3.0379,9.4175,11.5440,120.3008,80.2005,327.1816,748.8418,227.8423,

In [28]:
for(i in 1:ncol(mRNA_f)-1){
    if(is.na(mRNA_f[502,i+1])==TRUE){
        print(1)
        mRNA_f <- mRNA_f[,-(i+1)]
    }
}

In [29]:
tail(mRNA_f)

,name,ABAT,ABCA8,ABCB1,ADAMTS9,ADAMTSL2,AGPAT5,ALG1,ANK3,APLNR,APLP2,ARAP3,ARHGAP31,ARMCX2,AXIN2,B4GALT6,BAIAP3,BCAS3,BCL6B,BMPR2,BOC,CA5B,CA7,CALCRL,CCDC89,CCND2,CD34,CD44,CD99L2,CD9,CHRNB1,COL15A1,CTSO,CYB561,CYYR1,DOK6,DUOX2,DYNC1I1,DYSF,EBF2,EBF3,EDNRB,EIF4E3,ELOVL2,EMCN,ENG,ETS1,EXTL2,FBXL22,FRMD8,FSIP1,FZD3,FZD4,GFOD2,GIMAP6,GIPC3,GJC1,GNRHR2,GPR158,GPR176,GRB10,HECW2,HEY2,HIST1H2AC,HIST1H4H,HTR1F,IGDCC4,IKZF2,ITM2A,ITM2B,JAZF1,KCNK17,KCNK5,KCTD2,LANCL3,LMAN2L,LPCAT2,LZTFL1,MAGEH1,MCF2L,MMRN2,MOCS1,MTSS1,NAP1L2,NRIP2,NXPH3,OAZ2,ORMDL3,PBXIP1,PCDH12,PCDHB15,PDE8A,PDGFD,PIK3R3,PLCL1,PLEKHA1,PODXL,PRCP,PRR15,PTPRN2,QPCT,RALB,RAPGEF4,RFTN2,RGS5,RIMKLB,RNF146,RNF38,ROBO4,RUNDC3B,SCARF1,SCD5,SCN4A,SCNN1G,SERHL,SFTA3,SH3BGRL,SLIT3,SMG6,SPPL3,SPTLC1,ST6GALNAC6,STARD8,SULT1C4,TBC1D13,TCEAL8,TCF4,TEK,TESC,THSD7B,THUMPD1,TM4SF18,TMBIM6,TMEM204,TMEM57,TMEM59,TRIM4,TRPC4,UBL3,ZEB1,ZNF287,ZNF423,ZNF436,ZNF624,ZNF709,ZNF772,ADGRL2,ADGRL4,C10ORF76,C15ORF56,C1ORF95,C6ORF89,CEP126,GALNT16,PDIA3P1,PRR29,RMDN2,TMEM242,TMEM248
497,TCGA.MK.A4N6.01,1670.372,66.3089,32.9628,779.609,108.8540,551.5523,272.7635,798.7735,370.2568,89643.16,302.0314,495.2089,3083.174,140.2836,108.4707,882.7597,444.2315,236.8724,2629.360,307.0142,158.6585,5.3660,391.3377,1.9164,4835.186,599.4634,7909.544,635.8758,13537.371,505.1476,1111.920,1211.959,921.8091,180.9122,15.3315,4437.716,18.3979,1490.993,6.5159,7.6658,513.6067,1042.5450,23.3806,124.9521,1741.280,3043.695,209.2756,18.0146,1128.4017,5.3660,135.3009,357.6888,251.0540,197.3936,72.8248,111.5370,183.7754,1.9164,59.4097,918.3595,92.7558,146.0330,1261.4028,13.4151,1.9164,137.2173,1058.260,431.9663,16394.02,384.4385,2.6830,1120.3526,1033.729,11.4987,424.3005,1741.664,718.2829,466.4622,451.5140,471.0617,341.9433,172.0966,150.2491,38.3289,66.3089,1638.942,3599.847,6106.171,226.5236,18.0146,377.9226,169.4136,313.9134,57.8766,614.0284,1477.961,1490.993,602.9130,613.6451,2626.6769,1704.101,29.1299,29.8965,2922.576,612.7597,847.8344,1306.248,138.3672,10.7321,217.7079,2559.218,164.4308,69.7585,37.9456,1753.618,2629.360,1408.586,1524.339,1170.180,2376.006,1529.705,160.5979,20.6976,746.6462,1018.398,685.320,216.1748,523.5722,5.3660,828.670,17.6313,25183.21,170.9467,653.8904,7188.195,838.6355,13.0318,2160.981,378.3174,81.2572,62.4760,618.5473,58.6432,103.4879,248.3710,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
498,TCGA.MK.A4N7.01,1559.657,551.0747,33.4169,1564.517,169.2109,845.1432,471.3207,292.2458,570.5172,91345.33,512.7971,912.8883,5739.804,483.3295,293.7647,2353.3622,952.9885,267.9426,2433.964,1827.5993,513.2376,47.0874,645.8571,1.8227,7019.367,868.2312,12279.487,1099.1114,16588.441,743.9690,1087.871,1909.622,1005.5442,299.8405,37.6699,16243.336,43.7457,1531.404,24.6070,48.9102,540.7458,2054.2265,32.8093,520.0881,2757.497,2499.886,328.7005,53.7131,1506.4935,18.5312,178.3246,640.3038,360.5985,251.5379,119.6932,177.7170,576.5201,16.1009,91.4407,1288.0687,184.7042,369.7122,1566.3401,57.7201,7.5947,210.8301,3221.994,875.5221,18019.59,606.9720,65.6186,970.0008,1252.525,10.3289,535.2776,3016.329,985.7978,701.4506,580.8461,739.1205,487.5218,1308.7203,185.9193,111.7946,57.1125,2256.247,6075.796,12112.098,488.7978,262.4744,426.2171,405.2556,462.3680,81.1119,598.4659,1631.959,1617.681,3927.6980,678.3626,3301.2835,1480.368,131.8448,27.6449,7880.914,718.9124,2946.4571,1819.093,411.3314,8.2023,537.7079,4123.642,222.0703,531.0245,113.3136,4180.582,2994.152,5375.560,2347.384,1541.126,2640.845,1751.652,208.7036,27.0373,1223.9690,1181.742,1250.095,318.3717,6291.1825,25.8221,1180.831,73.2133,31316.78,290.4230,955.4189,9488.874,1119.7691,22.1767,3363.864,266.4024,194.7292,92.0483,791.6853,114.5287,171.6412,444.7482,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
499,TCGA.MK.A4N9.01,1556.532,411.9601,63.4444,5440.789,428.6786,531.5615,504.8805,1155.7175,273.9256,97369.63,798.1995,1418.9262,6733.255,434.2514,414.1035,1658.6389,488.2649,303.9331,2044.368,1685.9929,430.2861,64.3018,868.0742,2.5721,12895.081,1353.7670,12876.648,1088.4150,15142.214,662.2699

In [30]:
mRNA <- mRNA_f[1:501,1:(ncol(mRNA_f)-13)]

In [31]:
tail(mRNA)

,name,ABAT,ABCA8,ABCB1,ADAMTS9,ADAMTSL2,AGPAT5,ALG1,ANK3,APLNR,APLP2,ARAP3,ARHGAP31,ARMCX2,AXIN2,B4GALT6,BAIAP3,BCAS3,BCL6B,BMPR2,BOC,CA5B,CA7,CALCRL,CCDC89,CCND2,CD34,CD44,CD99L2,CD9,CHRNB1,COL15A1,CTSO,CYB561,CYYR1,DOK6,DUOX2,DYNC1I1,DYSF,EBF2,EBF3,EDNRB,EIF4E3,ELOVL2,EMCN,ENG,ETS1,EXTL2,FBXL22,FRMD8,FSIP1,FZD3,FZD4,GFOD2,GIMAP6,GIPC3,GJC1,GNRHR2,GPR158,GPR176,GRB10,HECW2,HEY2,HIST1H2AC,HIST1H4H,HTR1F,IGDCC4,IKZF2,ITM2A,ITM2B,JAZF1,KCNK17,KCNK5,KCTD2,LANCL3,LMAN2L,LPCAT2,LZTFL1,MAGEH1,MCF2L,MMRN2,MOCS1,MTSS1,NAP1L2,NRIP2,NXPH3,OAZ2,ORMDL3,PBXIP1,PCDH12,PCDHB15,PDE8A,PDGFD,PIK3R3,PLCL1,PLEKHA1,PODXL,PRCP,PRR15,PTPRN2,QPCT,RALB,RAPGEF4,RFTN2,RGS5,RIMKLB,RNF146,RNF38,ROBO4,RUNDC3B,SCARF1,SCD5,SCN4A,SCNN1G,SERHL,SFTA3,SH3BGRL,SLIT3,SMG6,SPPL3,SPTLC1,ST6GALNAC6,STARD8,SULT1C4,TBC1D13,TCEAL8,TCF4,TEK,TESC,THSD7B,THUMPD1,TM4SF18,TMBIM6,TMEM204,TMEM57,TMEM59,TRIM4,TRPC4,UBL3,ZEB1,ZNF287,ZNF423,ZNF436,ZNF624,ZNF709,ZNF772
496,TCGA.L6.A4EU.01,1199.796,47.1650,80.6463,270.4709,281.8255,410.8014,534.1699,847.5144,1270.8348,56098.99,331.3196,712.4245,1999.854,70.7475,107.7225,446.2071,349.0793,272.5089,2099.134,328.1170,146.5900,6.4051,419.8268,5.2406,6054.007,1362.5446,16966.591,965.4269,11230.220,472.2323,1667.953,1174.176,1187.2771,268.1418,25.6205,1202.125,25.3294,896.7174,5.5317,48.9119,196.5209,1110.7067,13.9748,390.1303,4128.682,2129.122,229.1288,13.8118,797.7291,9.8988,80.0641,618.1440,255.3315,251.2555,97.2414,110.3428,624.3424,3.2026,19.5065,741.2475,59.9753,592.7651,1352.3546,18.3419,3.2026,78.6083,592.7651,306.2814,20155.18,344.4210,5.2406,1389.6208,1022.782,4.0760,572.9675,1601.863,701.0699,429.7256,330.4782,727.2727,246.1635,327.8259,100.4440,82.6843,36.9750,1787.030,3193.828,7974.088,383.1429,129.5582,341.2184,259.4075,237.8630,79.4818,517.9416,1566.053,1774.802,2462.4791,623.3350,2408.6178,1403.013,87.6337,43.6713,4607.322,411.1333,1198.6316,869.350,441.0801,29.6965,446.9030,5230.366,44.2536,98.9883,49.4941,4911.371,2863.964,706.8928,1239.974,1455.419,1952.107,2542.252,94.6212,6.6963,1141.8589,1227.164,807.9191,239.9010,882.4514,20.9622,1006.187,28.8231,22123.59,259.1164,770.6529,7864.619,883.9071,10.4811,1882.815,303.0846,88.2160,86.1780,398.8645,72.4944,94.3300,266.9772
497,TCGA.MK.A4N6.01,1670.372,66.3089,32.9628,779.6090,108.8540,551.5523,272.7635,798.7735,370.2568,89643.16,302.0314,495.2089,3083.174,140.2836,108.4707,882.7597,444.2315,236.8724,2629.360,307.0142,158.6585,5.3660,391.3377,1.9164,4835.186,599.4634,7909.544,635.8758,13537.371,505.1476,1111.920,1211.959,921.8091,180.9122,15.3315,4437.716,18.3979,1490.9927,6.5159,7.6658,513.6067,1042.5450,23.3806,124.9521,1741.280,3043.695,209.2756,18.0146,1128.4017,5.3660,135.3009,357.6888,251.0540,197.3936,72.8248,111.5370,183.7754,1.9164,59.4097,918.3595,92.7558,146.0330,1261.4028,13.4151,1.9164,137.2173,1058.2599,431.9663,16394.02,384.4385,2.6830,1120.3526,1033.729,11.4987,424.3005,1741.664,718.2829,466.4622,451.5140,471.0617,341.9433,172.0966,150.2491,38.3289,66.3089,1638.942,3599.847,6106.171,226.5236,18.0146,377.9226,169.4136,313.9134,57.8766,614.0284,1477.961,1490.993,602.9130,613.6451,2626.6769,1704.101,29.1299,29.8965,2922.576,612.7597,847.8344,1306.248,138.3672,10.7321,217.7079,2559.218,164.4308,69.7585,37.9456,1753.618,2629.360,1408.5857,1524.339,1170.180,2376.006,1529.705,160.5979,20.6976,746.6462,1018.398,685.3200,216.1748,523.5722,5.3660,828.670,17.6313,25183.21,170.9467,653.8904,7188.195,838.6355,13.0318,2160.981,378.3174,81.2572,62.4760,618.5473,58.6432,103.4879,248.3710
498,TCGA.MK.A4N7.01,1559.657,551.0747,33.4169,1564.5174,169.2109,845.1432,471.3207,292.2458,570.5172,91345.33,512.7971,912.8883,5739.804,483.3295,293.7647,2353.3622,952.9885,267.9426,2433.964,1827.5993,513.2376,47.0874,645.8571,1.8227,7019.367,868.2312,12279.487,1099.1114,16588.441,743.9690,1087.871,1909.622,1005.5442,299.8405,37.6699,16243.336,43.7457,1531.4043,24.6070,48.9102,540.7458,2054.2265,32.8093,520.0881,2757.497,2499.886,328.7005,53.7131,1506.4935,18.5312,178.3246,640.3038,360.598

In [32]:
mrna_p_uni <- merge(clinical_p_uni, mRNA, by = "name")
mrna_p_multi <- merge(clinical_p_multi, mRNA, by = "name")
tail(mrna_p_uni, n=20L)
tail(mrna_p_multi, n=20L)

,name,multifocality,BRAF,focality,ABAT,ABCA8,ABCB1,ADAMTS9,ADAMTSL2,AGPAT5,ALG1,ANK3,APLNR,APLP2,ARAP3,ARHGAP31,ARMCX2,AXIN2,B4GALT6,BAIAP3,BCAS3,BCL6B,BMPR2,BOC,CA5B,CA7,CALCRL,CCDC89,CCND2,CD34,CD44,CD99L2,CD9,CHRNB1,COL15A1,CTSO,CYB561,CYYR1,DOK6,DUOX2,DYNC1I1,DYSF,EBF2,EBF3,EDNRB,EIF4E3,ELOVL2,EMCN,ENG,ETS1,EXTL2,FBXL22,FRMD8,FSIP1,FZD3,FZD4,GFOD2,GIMAP6,GIPC3,GJC1,GNRHR2,GPR158,GPR176,GRB10,HECW2,HEY2,HIST1H2AC,HIST1H4H,HTR1F,IGDCC4,IKZF2,ITM2A,ITM2B,JAZF1,KCNK17,KCNK5,KCTD2,LANCL3,LMAN2L,LPCAT2,LZTFL1,MAGEH1,MCF2L,MMRN2,MOCS1,MTSS1,NAP1L2,NRIP2,NXPH3,OAZ2,ORMDL3,PBXIP1,PCDH12,PCDHB15,PDE8A,PDGFD,PIK3R3,PLCL1,PLEKHA1,PODXL,PRCP,PRR15,PTPRN2,QPCT,RALB,RAPGEF4,RFTN2,RGS5,RIMKLB,RNF146,RNF38,ROBO4,RUNDC3B,SCARF1,SCD5,SCN4A,SCNN1G,SERHL,SFTA3,SH3BGRL,SLIT3,SMG6,SPPL3,SPTLC1,ST6GALNAC6,STARD8,SULT1C4,TBC1D13,TCEAL8,TCF4,TEK,TESC,THSD7B,THUMPD1,TM4SF18,TMBIM6,TMEM204,TMEM57,TMEM59,TRIM4,TRPC4,UBL3,ZEB1,ZNF287,ZNF423,ZNF436,ZNF624,ZNF709,ZNF772
108,TCGA.FK.A3SB.01,unifocal,1,0,3338.4247,511.5304,182.3899,4781.0722,627.4334,1019.4669,696.3702,2350.1048,2779.8742,129834.98,893.0818,2585.5046,6916.742,231.5064,336.6277,1171.8269,474.9925,997.0051,2979.934,2466.6068,426.7984,26.0557,2005.6903,10.1827,6952.980,3331.5364,11514.825,1248.2779,16941.899,650.4852,2769.9910,1386.3432,1451.0332,877.8077,29.6496,20179.095,44.6241,2244.3845,77.2686,194.6691,1272.5367,1245.5825,70.3804,1026.0557,9578.913,7244.085,410.0030,50.7008,1150.0449,8.9847,168.9128,1385.6095,715.4837,967.3555,396.2264,408.2060,484.8907,27.5532,148.5475,2709.1944,504.9416,1085.9539,1631.0273,73.3753,10.7817,221.0243,2120.9943,1398.6223,21856.84,515.4238,54.8068,2156.0347,1257.2627,11.6801,858.3408,4587.901,1001.4975,811.6202,987.7209,2226.7146,696.9692,2188.3768,359.3890,276.7296,101.5274,2831.686,8936.807,14004.492,1677.4483,78.4666,511.2309,1229.7095,921.5334,163.5220,716.9811,4808.925,2594.489,3378.2570,772.3869,4484.8757,1510.033,444.4444,60.1977,18682.240,780.1048,1952.6804,1469.3022,1580.7128,47.9185,1021.8628,4938.604,496.2564,623.8395,139.5627,5890.913,2200.925,7991.015,1820.3055,1910.752,2716.083,2641.8089,429.4699,51.8119,1118.5984,1255.1662,1837.3765,995.5076,3749.3261,54.5073,1183.5879,239.2932,29472.00,601.9766,1035.3399,9992.513,861.3357,49.4160,2787.661,829.7245,176.6996,221.6232,789.5058,151.5424,145.8521,443.2465
109,TCGA.FY.A3I4.01,unifocal,1,0,1727.5530,528.3662,149.5376,531.7767,643.0117,374.1064,436.0254,1518.4626,1293.8939,77318.29,660.5890,1569.6203,2253.033,602.0857,44.5989,405.2023,457.2703,477.9957,1848.757,2715.5506,483.6387,3.1482,1114.1864,3.4105,9976.258,3016.1999,6989.440,1225.6837,8706.237,425.4844,3204.0401,916.6393,2150.7182,505.0174,45.3860,7351.479,25.1853,1402.2431,61.6515,179.1828,631.9932,729.0615,28.8581,631.2061,5650.161,3673.903,230.3404,56.5698,1035.2200,7.6081,105.4634,1502.8110,579.5238,616.2524,119.1054,224.8311,495.8877,16.2576,123.5653,1335.0823,390.8966,238.2108,965.6982,49.3212,3.6729,427.6251,685.5119,946.5469,13534.47,397.4552,41.4508,800.4198,1338.4928,13.3797,710.4348,3040.598,518.6594,513.9372,671.2639,1335.8694,757.3136,508.1655,155.8339,155.8339,138.5191,2137.076,2642.093,10981.833,702.0397,58.2410,502.9186,601.5610,1019.2169,145.3401,626.2216,4788.877,2012.986,1033.1213,671.6810,300.1246,1195.514,178.1334,57.9786,5520.037,577.3123,877.5497,1031.8095,1186.0694,24.6606,552.5021,1724.929,541.4836,313.7666,86.5744,2828.919,1715.469,3045.058,2054.6993,1732.013,1575.654,1481.9965,509.2149,30.1699,1471.7649,823.5063,1370.7615,581.0979,551.7151,19.9383,972.5192,157.1457,21593.49,732.9967,934.4789,6596.708,991.4081,11.5433,1628.124,501.7328,146.6518,311.4055,711.5839,81.0651,133.7968,246.6059
110,TCGA.FY.A3NN.01,unifocal,1,0,1422.9381,376.1918,71.0914,2296.3991,250.3008,587.9848,530.5045,498.0098,381.0053,73149.68,309.5436,1689.1604,6440.433,233.2685,283.6249,137.0582,383.2269,178.8392,2588.540,193.2796,262.8270,25.9187,674.2572,2.2216,10541.516,434.6941,12734.981,848.2829,10212.719,659.8167,559.1040,1339.6279,1653.2445,30

,name,multifocality,BRAF,focality,ABAT,ABCA8,ABCB1,ADAMTS9,ADAMTSL2,AGPAT5,ALG1,ANK3,APLNR,APLP2,ARAP3,ARHGAP31,ARMCX2,AXIN2,B4GALT6,BAIAP3,BCAS3,BCL6B,BMPR2,BOC,CA5B,CA7,CALCRL,CCDC89,CCND2,CD34,CD44,CD99L2,CD9,CHRNB1,COL15A1,CTSO,CYB561,CYYR1,DOK6,DUOX2,DYNC1I1,DYSF,EBF2,EBF3,EDNRB,EIF4E3,ELOVL2,EMCN,ENG,ETS1,EXTL2,FBXL22,FRMD8,FSIP1,FZD3,FZD4,GFOD2,GIMAP6,GIPC3,GJC1,GNRHR2,GPR158,GPR176,GRB10,HECW2,HEY2,HIST1H2AC,HIST1H4H,HTR1F,IGDCC4,IKZF2,ITM2A,ITM2B,JAZF1,KCNK17,KCNK5,KCTD2,LANCL3,LMAN2L,LPCAT2,LZTFL1,MAGEH1,MCF2L,MMRN2,MOCS1,MTSS1,NAP1L2,NRIP2,NXPH3,OAZ2,ORMDL3,PBXIP1,PCDH12,PCDHB15,PDE8A,PDGFD,PIK3R3,PLCL1,PLEKHA1,PODXL,PRCP,PRR15,PTPRN2,QPCT,RALB,RAPGEF4,RFTN2,RGS5,RIMKLB,RNF146,RNF38,ROBO4,RUNDC3B,SCARF1,SCD5,SCN4A,SCNN1G,SERHL,SFTA3,SH3BGRL,SLIT3,SMG6,SPPL3,SPTLC1,ST6GALNAC6,STARD8,SULT1C4,TBC1D13,TCEAL8,TCF4,TEK,TESC,THSD7B,THUMPD1,TM4SF18,TMBIM6,TMEM204,TMEM57,TMEM59,TRIM4,TRPC4,UBL3,ZEB1,ZNF287,ZNF423,ZNF436,ZNF624,ZNF709,ZNF772
91,TCGA.ET.A39O.01,multifocal,1,1,1773.9348,12.9180,32.9201,438.7957,160.8501,509.6364,506.3486,917.5956,453.7973,76030.42,226.6903,761.7460,2900.719,82.9253,137.0976,837.8914,365.8714,96.6767,2751.953,212.1054,144.9359,21.6689,481.7168,10.8345,7549.536,616.7309,13473.904,895.5099,11292.010,547.1403,1008.8551,1402.6461,1228.4613,240.0250,21.2522,2321.492,26.2527,1166.7882,17.5018,35.8371,351.2866,1325.5547,20.8355,169.6010,2693.610,1883.530,244.6088,18.1602,842.1711,6.6674,125.8464,368.2634,159.1832,267.9446,69.5906,153.7660,306.3027,0.0000,76.2579,740.4938,88.3425,375.0391,982.6024,14.5849,16.2517,82.0919,1134.7015,229.6073,20149.18,520.8876,12.5013,1415.5641,1146.3694,4.5838,519.6375,2775.289,874.6744,542.9732,293.8348,485.0505,378.1852,204.1879,171.2678,62.9232,39.1707,1781.019,3365.767,6115.637,233.3576,15.8350,415.4599,198.7707,572.5596,65.8402,588.8113,1373.060,1512.241,896.7601,685.9048,3724.555,1763.9337,74.5911,25.8360,7114.491,353.3618,1020.9397,1055.1099,272.5284,13.3347,297.1143,3797.062,226.6903,81.6752,97.5102,2129.447,3026.149,1466.4028,1298.469,1335.556,2637.775,1632.253,147.0987,2.9170,981.7689,1261.7981,758.8290,173.3514,819.6687,6.6674,931.3470,48.7551,28788.83,186.2694,701.3231,8890.509,997.6039,12.9180,2442.755,225.0234,136.6809,49.5885,471.8075,92.0929,80.8418,374.6224
92,TCGA.ET.A3BO.01,multifocal,1,1,1873.0740,112.5693,113.2295,1900.4736,381.9434,612.3638,545.6937,1383.8432,940.8285,63855.18,488.9007,1568.0475,4437.739,472.7251,326.4840,804.7880,360.8160,255.5092,2797.397,806.8017,298.2657,77.5771,940.8285,7.9228,7626.653,1465.3816,17069.600,1103.9054,10044.417,625.5651,1936.1260,1442.9338,1351.1618,394.4877,25.7490,5932.171,43.5752,1025.3380,27.7297,31.0308,653.9583,1767.4371,35.6524,328.7948,4999.596,2862.760,210.2834,27.2510,698.1938,9.2432,134.3569,920.1303,380.6229,568.1284,156.8047,355.8643,440.2912,3.9614,139.6388,820.6665,233.7216,623.5877,1092.6815,39.2837,19.4768,165.7179,1907.7361,535.1168,17859.24,530.1651,17.1660,1416.5246,1207.2315,5.6120,601.4700,2938.686,1013.7840,589.5859,488.5706,731.5354,550.5794,609.7229,263.7621,126.4341,85.8300,2076.095,4901.551,7817.129,505.7366,62.0617,471.7347,278.2872,745.0701,106.2971,783.6936,3246.684,1453.497,633.1611,967.2377,2433.280,1323.1020,188.1657,34.6621,12229.450,486.4314,1414.2138,1616.5744,495.8331,32.6814,538.0879,4065.039,226.7892,323.5130,52.4883,6262.679,2569.617,2458.3683,1903.445,1653.217,1976.730,1779.982,282.9088,8.5830,1464.0612,1196.9979,1235.6214,419.9066,376.0013,15.1853,1534.3757,96.3937,22171.53,339.6886,887.0197,7127.519,1136.2567,24.0984,3429.898,402.9717,192.1271,86.4902,724.7284,117.1909,179.5827,504.4161
93,TCGA.ET.A3BP.01,multifocal,1,1,2261.0427,190.0797,144.4605,967.4149,778.7835,568.0666,795.6191,242.9399,1510.1376,104412.02,803.4033,1486.2419,6792.904,430.1231,289.2831,1325.4236,586.8936,540.9124,2271.542,526.7922,246.0246,72.7734,1518.1028,17.0167,17539.464,3297.6104,17457.277,1410.2100,11144.823,1219.3773,1437.7263,1422.8820,2990.2245,828.7473,52.4982,14893.193,110.0652,1595.9450,47.

In [33]:
mrna_p_uni <- mrna_p_uni[5:ncol(mrna_p_uni)]
mrna_p_multi <- mrna_p_multi[5:ncol(mrna_p_multi)]

---
### loading miRNA

In [34]:
load("miRNA.RData")

In [35]:
#miRNA df name col 분리
name <- rownames(miRNA)
rownames(miRNA) <- NULL
miRNA <- cbind(name,miRNA)

In [36]:
#miRNA name 형식 변경
#TCGA-DJ-A2Q5-01A-11R-A18B-13 type >>> TCGA.BJ.A0YZ.01 type(clinical2)
miRNA$name <- gsub("-", ".", miRNA$name)
miRNA$name <- substr(miRNA$name, 1, 15)

In [37]:
count.zero <- sapply(1:ncol(miRNA), function(i) sum(miRNA[, i] == 0)) 
selected.mirna <- names(miRNA)[which(count.zero/nrow(miRNA) < 0.1)]
del.mirna <- setdiff(names(miRNA), selected.mirna)

In [38]:
mirna <- miRNA[, selected.mirna]
#mirna[mirna == 0] <- NA

In [39]:
head(mirna)

name,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,hsa-let-7i,hsa-mir-1-2,hsa-mir-100,hsa-mir-101-1,hsa-mir-101-2,hsa-mir-103-1,hsa-mir-103-2,hsa-mir-106a,hsa-mir-106b,hsa-mir-107,hsa-mir-10a,hsa-mir-10b,hsa-mir-1179,hsa-mir-1180,hsa-mir-1224,hsa-mir-1226,hsa-mir-1247,hsa-mir-1248,hsa-mir-1249,hsa-mir-1251,hsa-mir-1254,hsa-mir-125a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1266,hsa-mir-127,hsa-mir-1270-1,hsa-mir-1270-2,hsa-mir-1271,hsa-mir-1274b,hsa-mir-1277,hsa-mir-128-1,hsa-mir-128-2,hsa-mir-1287,hsa-mir-1288,hsa-mir-129-1,hsa-mir-129-2,hsa-mir-1291,hsa-mir-1292,hsa-mir-1296,hsa-mir-1301,hsa-mir-1306,hsa-mir-1307,hsa-mir-130a,hsa-mir-130b,hsa-mir-132,hsa-mir-133a-1,hsa-mir-133b,hsa-mir-134,hsa-mir-135a-1,hsa-mir-135a-2,hsa-mir-135b,hsa-mir-136,hsa-mir-138-1,hsa-mir-138-2,hsa-mir-139,hsa-mir-140,hsa-mir-141,hsa-mir-142,hsa-mir-143,hsa-mir-144,hsa-mir-145,hsa-mir-1468,hsa-mir-146a,hsa-mir-146b,hsa-mir-148a,hsa-mir-148b,hsa-mir-149,hsa-mir-150,hsa-mir-151,hsa-mir-152,hsa-mir-153-2,hsa-mir-155,hsa-mir-15a,hsa-mir-15b,hsa-mir-16-1,hsa-mir-16-2,hsa-mir-17,hsa-mir-181a-1,hsa-mir-181a-2,hsa-mir-181b-1,hsa-mir-181b-2,hsa-mir-181c,hsa-mir-181d,hsa-mir-182,hsa-mir-183,hsa-mir-185,hsa-mir-186,...,hsa-mir-423,hsa-mir-424,hsa-mir-425,hsa-mir-429,hsa-mir-431,hsa-mir-4326,hsa-mir-450a-1,hsa-mir-450a-2,hsa-mir-450b,hsa-mir-451,hsa-mir-452,hsa-mir-454,hsa-mir-455,hsa-mir-484,hsa-mir-486,hsa-mir-491,hsa-mir-493,hsa-mir-495,hsa-mir-497,hsa-mir-499,hsa-mir-500a,hsa-mir-500b,hsa-mir-501,hsa-mir-502,hsa-mir-503,hsa-mir-505,hsa-mir-508,hsa-mir-509-1,hsa-mir-509-2,hsa-mir-509-3,hsa-mir-511-1,hsa-mir-511-2,hsa-mir-514-1,hsa-mir-514-2,hsa-mir-514-3,hsa-mir-532,hsa-mir-542,hsa-mir-548b,hsa-mir-548v,hsa-mir-550a-1,hsa-mir-550a-2,hsa-mir-551b,hsa-mir-574,hsa-mir-576,hsa-mir-577,hsa-mir-582,hsa-mir-584,hsa-mir-585,hsa-mir-589,hsa-mir-590,hsa-mir-592,hsa-mir-598,hsa-mir-616,hsa-mir-624,hsa-mir-625,hsa-mir-627,hsa-mir-628,hsa-mir-629,hsa-mir-639,hsa-mir-642a,hsa-mir-643,hsa-mir-651,hsa-mir-652,hsa-mir-653,hsa-mir-654,hsa-mir-659,hsa-mir-660,hsa-mir-664,hsa-mir-671,hsa-mir-675,hsa-mir-676,hsa-mir-7-1,hsa-mir-7-2,hsa-mir-708,hsa-mir-744,hsa-mir-758,hsa-mir-760,hsa-mir-766,hsa-mir-769,hsa-mir-874,hsa-mir-877,hsa-mir-887,hsa-mir-889,hsa-mir-891a,hsa-mir-9-1,hsa-mir-9-2,hsa-mir-92a-1,hsa-mir-92a-2,hsa-mir-92b,hsa-mir-93,hsa-mir-935,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
TCGA.4C.A93U.01,5954.66,11895.20,6031.335,5887.396,2471.328,730.7689,2358.667,25.18938,9810.296,923.4262,3680.6949,4.705709,7469.898,13082.148,118.19634,27710.54,7.196967,6.366548,355.9731,56.46851,6056.801,50894.18,0.000000,42.90499,0.000000,2.768064,2.768064,2.491258,1.660839,34.87761,0.276806,1930.7248,3773.425,27.68064,6794.214,8.857805,685.372691,1.107226,0.000000,3.598483,0.553613,2.768064,62.28144,39.30651,15.77797,0.553613,0.553613,0.830419,2.768064,1.384032,4.428903,9.965031,4.152096,1142.3801,194.87172,16.885191,68.09438,5.536128,0.553613,149.752272,536.7276,24.08216,692.8465,34.600802,12.733095,9.688225,93.00696,839.5539,1630.943,1777.6508,35985.94,145.87698,1327.8404,1.107226,173.00401,61028.0646,28806.97,192.9341,5.536128,288.98590,2021.240,193.2109,3.598483,348.77609,435.9701,301.4422,1356.6283,13.840321,552.7824,7360.006,2287.805,1135.7367,16.885191,113.76744,22.42132,14914.330,6181.641,69.75522,297.0133,...,175.7721,88.57805,480.53594,130.65263,6.920160,3.875290,3.044871,3.044871,8.027386,1155.3900,37.92248,11.902676,82.21151,63.388670,145.60018,2.214451,13.563514,7.750580,13.840321,0.276806,174.94166,6.920160,25.74300,4.982516,6.643354,26.85022,29.618287,4.152096,4.152096,5.536128,14.117127,15.224353,3.321677,6.366548,6.089741,610.0813,182.6922,1.107226,0.830419,2.491258,0.553613,42.074576,110.44576,11.349063,0.553613,69.47841,64.77270,0.000000,34.32400,20.483675,12.179482,62.55825,1.937645,0.830419,98.81989,1.937645,13.00990,52.31641

In [40]:
p_uni_miRNA <- merge(clinical_p_uni, mirna, by.x = "name", by.y = "name", all.x = TRUE)
p_multi_miRNA <- merge(clinical_p_multi, mirna, by.x = "name", by.y = "name", all.x = TRUE)
tail(p_uni_miRNA, n=20L)
tail(p_multi_miRNA, n=20L)

,name,multifocality,BRAF,focality,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,hsa-let-7i,hsa-mir-1-2,hsa-mir-100,hsa-mir-101-1,hsa-mir-101-2,hsa-mir-103-1,hsa-mir-103-2,hsa-mir-106a,hsa-mir-106b,hsa-mir-107,hsa-mir-10a,hsa-mir-10b,hsa-mir-1179,hsa-mir-1180,hsa-mir-1224,hsa-mir-1226,hsa-mir-1247,hsa-mir-1248,hsa-mir-1249,hsa-mir-1251,hsa-mir-1254,hsa-mir-125a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1266,hsa-mir-127,hsa-mir-1270-1,hsa-mir-1270-2,hsa-mir-1271,hsa-mir-1274b,hsa-mir-1277,hsa-mir-128-1,hsa-mir-128-2,hsa-mir-1287,hsa-mir-1288,hsa-mir-129-1,hsa-mir-129-2,hsa-mir-1291,hsa-mir-1292,hsa-mir-1296,hsa-mir-1301,hsa-mir-1306,hsa-mir-1307,hsa-mir-130a,hsa-mir-130b,hsa-mir-132,hsa-mir-133a-1,hsa-mir-133b,hsa-mir-134,hsa-mir-135a-1,hsa-mir-135a-2,hsa-mir-135b,hsa-mir-136,hsa-mir-138-1,hsa-mir-138-2,hsa-mir-139,hsa-mir-140,hsa-mir-141,hsa-mir-142,hsa-mir-143,hsa-mir-144,hsa-mir-145,hsa-mir-1468,hsa-mir-146a,hsa-mir-146b,hsa-mir-148a,hsa-mir-148b,hsa-mir-149,hsa-mir-150,hsa-mir-151,hsa-mir-152,hsa-mir-153-2,hsa-mir-155,hsa-mir-15a,hsa-mir-15b,hsa-mir-16-1,hsa-mir-16-2,hsa-mir-17,hsa-mir-181a-1,hsa-mir-181a-2,hsa-mir-181b-1,hsa-mir-181b-2,hsa-mir-181c,hsa-mir-181d,hsa-mir-182,...,hsa-mir-423,hsa-mir-424,hsa-mir-425,hsa-mir-429,hsa-mir-431,hsa-mir-4326,hsa-mir-450a-1,hsa-mir-450a-2,hsa-mir-450b,hsa-mir-451,hsa-mir-452,hsa-mir-454,hsa-mir-455,hsa-mir-484,hsa-mir-486,hsa-mir-491,hsa-mir-493,hsa-mir-495,hsa-mir-497,hsa-mir-499,hsa-mir-500a,hsa-mir-500b,hsa-mir-501,hsa-mir-502,hsa-mir-503,hsa-mir-505,hsa-mir-508,hsa-mir-509-1,hsa-mir-509-2,hsa-mir-509-3,hsa-mir-511-1,hsa-mir-511-2,hsa-mir-514-1,hsa-mir-514-2,hsa-mir-514-3,hsa-mir-532,hsa-mir-542,hsa-mir-548b,hsa-mir-548v,hsa-mir-550a-1,hsa-mir-550a-2,hsa-mir-551b,hsa-mir-574,hsa-mir-576,hsa-mir-577,hsa-mir-582,hsa-mir-584,hsa-mir-585,hsa-mir-589,hsa-mir-590,hsa-mir-592,hsa-mir-598,hsa-mir-616,hsa-mir-624,hsa-mir-625,hsa-mir-627,hsa-mir-628,hsa-mir-629,hsa-mir-639,hsa-mir-642a,hsa-mir-643,hsa-mir-651,hsa-mir-652,hsa-mir-653,hsa-mir-654,hsa-mir-659,hsa-mir-660,hsa-mir-664,hsa-mir-671,hsa-mir-675,hsa-mir-676,hsa-mir-7-1,hsa-mir-7-2,hsa-mir-708,hsa-mir-744,hsa-mir-758,hsa-mir-760,hsa-mir-766,hsa-mir-769,hsa-mir-874,hsa-mir-877,hsa-mir-887,hsa-mir-889,hsa-mir-891a,hsa-mir-9-1,hsa-mir-9-2,hsa-mir-92a-1,hsa-mir-92a-2,hsa-mir-92b,hsa-mir-93,hsa-mir-935,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
108,TCGA.FK.A3SB.01,unifocal,1,0,23779.012,47652.19,23719.517,12255.651,4728.095,956.6679,5669.294,84.48187,54920.525,2035.0442,2091.479,32.296892,11266.176,23587.44,80.91221,19612.88,6.799346,9.859051,330.9582,91.11123,1730.0935,8788.834,2.379771,31.61696,1.529853,0.339967,7.819248,1.359869,0.000000,156.89490,0.339967,1452.340,2692.711,36.54648,9203.424,5.609460,127.317748,2.549755,2.719738,1.699836,1.189885,1.869820,85.67176,60.51418,40.62609,0.000000,0.509951,0.509951,0.509951,0.679935,4.929526,7.649264,11.218920,873.5459,217.57906,8.669166,251.57579,25.667530,5.609460,30.087105,673.4752,118.81857,220.8088,11.558888,15.298528,11.048937,168.62377,1842.6227,1471.7184,444.5072,126734.87,368.69452,2040.8236,5.439477,66.12364,57278.368,13702.72,350.8462,6.799346,97.40063,3068.205,197.35101,7.649264,116.09883,175.4231,134.79703,459.4658,4.929526,615.6808,8262.565,4666.561,1786.0181,26.517448,483.7734,75.30275,3332.019,...,193.4414,611.43116,98.08056,147.03585,1.189885,2.719738,29.747137,30.257088,143.29621,1019.2219,76.83261,9.179117,120.00845,37.39640,238.14708,5.609460,3.569656,1.699836,23.797710,1.019902,107.93961,3.059706,18.52822,8.159215,120.518403,19.20815,71.05316,6.289395,5.949427,8.329198,2.719738,3.229689,10.029035,10.708969,8.159215,789.0641,2313.30739,4.079607,1.359869,1.699836,0.509951,67.31352,54.22478,7.139313,0.509951,45.89558,87.88154,0.679935,32.12691,15.298528,2.549755,86.18171,3.569656,2.039804,134.62705,0.509951,15.12854,66.12364,0.339967,

,name,multifocality,BRAF,focality,hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,hsa-let-7i,hsa-mir-1-2,hsa-mir-100,hsa-mir-101-1,hsa-mir-101-2,hsa-mir-103-1,hsa-mir-103-2,hsa-mir-106a,hsa-mir-106b,hsa-mir-107,hsa-mir-10a,hsa-mir-10b,hsa-mir-1179,hsa-mir-1180,hsa-mir-1224,hsa-mir-1226,hsa-mir-1247,hsa-mir-1248,hsa-mir-1249,hsa-mir-1251,hsa-mir-1254,hsa-mir-125a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1266,hsa-mir-127,hsa-mir-1270-1,hsa-mir-1270-2,hsa-mir-1271,hsa-mir-1274b,hsa-mir-1277,hsa-mir-128-1,hsa-mir-128-2,hsa-mir-1287,hsa-mir-1288,hsa-mir-129-1,hsa-mir-129-2,hsa-mir-1291,hsa-mir-1292,hsa-mir-1296,hsa-mir-1301,hsa-mir-1306,hsa-mir-1307,hsa-mir-130a,hsa-mir-130b,hsa-mir-132,hsa-mir-133a-1,hsa-mir-133b,hsa-mir-134,hsa-mir-135a-1,hsa-mir-135a-2,hsa-mir-135b,hsa-mir-136,hsa-mir-138-1,hsa-mir-138-2,hsa-mir-139,hsa-mir-140,hsa-mir-141,hsa-mir-142,hsa-mir-143,hsa-mir-144,hsa-mir-145,hsa-mir-1468,hsa-mir-146a,hsa-mir-146b,hsa-mir-148a,hsa-mir-148b,hsa-mir-149,hsa-mir-150,hsa-mir-151,hsa-mir-152,hsa-mir-153-2,hsa-mir-155,hsa-mir-15a,hsa-mir-15b,hsa-mir-16-1,hsa-mir-16-2,hsa-mir-17,hsa-mir-181a-1,hsa-mir-181a-2,hsa-mir-181b-1,hsa-mir-181b-2,hsa-mir-181c,hsa-mir-181d,hsa-mir-182,...,hsa-mir-423,hsa-mir-424,hsa-mir-425,hsa-mir-429,hsa-mir-431,hsa-mir-4326,hsa-mir-450a-1,hsa-mir-450a-2,hsa-mir-450b,hsa-mir-451,hsa-mir-452,hsa-mir-454,hsa-mir-455,hsa-mir-484,hsa-mir-486,hsa-mir-491,hsa-mir-493,hsa-mir-495,hsa-mir-497,hsa-mir-499,hsa-mir-500a,hsa-mir-500b,hsa-mir-501,hsa-mir-502,hsa-mir-503,hsa-mir-505,hsa-mir-508,hsa-mir-509-1,hsa-mir-509-2,hsa-mir-509-3,hsa-mir-511-1,hsa-mir-511-2,hsa-mir-514-1,hsa-mir-514-2,hsa-mir-514-3,hsa-mir-532,hsa-mir-542,hsa-mir-548b,hsa-mir-548v,hsa-mir-550a-1,hsa-mir-550a-2,hsa-mir-551b,hsa-mir-574,hsa-mir-576,hsa-mir-577,hsa-mir-582,hsa-mir-584,hsa-mir-585,hsa-mir-589,hsa-mir-590,hsa-mir-592,hsa-mir-598,hsa-mir-616,hsa-mir-624,hsa-mir-625,hsa-mir-627,hsa-mir-628,hsa-mir-629,hsa-mir-639,hsa-mir-642a,hsa-mir-643,hsa-mir-651,hsa-mir-652,hsa-mir-653,hsa-mir-654,hsa-mir-659,hsa-mir-660,hsa-mir-664,hsa-mir-671,hsa-mir-675,hsa-mir-676,hsa-mir-7-1,hsa-mir-7-2,hsa-mir-708,hsa-mir-744,hsa-mir-758,hsa-mir-760,hsa-mir-766,hsa-mir-769,hsa-mir-874,hsa-mir-877,hsa-mir-887,hsa-mir-889,hsa-mir-891a,hsa-mir-9-1,hsa-mir-9-2,hsa-mir-92a-1,hsa-mir-92a-2,hsa-mir-92b,hsa-mir-93,hsa-mir-935,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
91,TCGA.ET.A39O.01,multifocal,1,1,25824.398,51508.16,25848.527,5191.350,2699.759,892.6381,7743.050,97.08796,70244.567,2836.1107,3347.536,13.420983,7512.467,13493.370,56.39668,17237.68,6.996044,2.569975,399.0601,80.95423,2826.545,12517.208,0.571106,22.84423,0.142776,0.856658,4.711622,3.712187,0.571106,93.23300,0.285553,1649.924,4251.310,42.97570,2725.602,12.849877,239.721597,4.426069,4.140516,2.712752,2.141646,1.713317,74.95762,58.68111,28.98361,0.428329,3.426634,3.283857,0.856658,0.571106,4.568845,11.136560,9.137690,1333.8173,110.50894,12.421548,55.25447,5.139951,0.571106,58.395553,427.1870,73.67263,209.1674,11.707666,21.702015,13.849312,94.94631,617.6508,1398.6377,2172.6287,62557.63,40.12017,673.9047,13.278206,260.28140,37715.25,31911.24,267.2774,9.566020,269.13354,3712.758,169.3328,3.997740,349.23111,201.88585,111.3656,683.1851,4.854398,479.0149,5167.078,5327.702,1126.5059,27.12752,177.3283,34.69467,5553.717,...,310.1104,225.58673,206.59747,93.946880,1.284988,3.997740,7.424373,5.568280,40.548501,104.5123,29.55472,7.567150,81.66811,33.40968,37.26464,7.567150,7.281597,1.713317,8.852138,0.999435,150.20079,3.141081,23.700885,6.853268,45.688452,21.41646,52.39894,5.853833,6.139386,15.848182,8.709361,7.709926,10.708231,10.565455,11.136560,778.2742,414.90826,1.713317,0.428329,2.284423,1.998870,43.54681,65.96270,9.994349,1.570541,33.40968,60.67998,0.571106,44.26069,16.133735,5.139951,48.54398,5.139951,0.285553,133.06762,0.713882,15.848182,64.53494,0.285553,1.998870

In [41]:
mirna_p_uni <- p_uni_miRNA[5:ncol(p_uni_miRNA)]
mirna_p_multi <- p_multi_miRNA[5:ncol(p_multi_miRNA)]
head(mirna_p_uni)

hsa-let-7a-1,hsa-let-7a-2,hsa-let-7a-3,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f-1,hsa-let-7f-2,hsa-let-7g,hsa-let-7i,hsa-mir-1-2,hsa-mir-100,hsa-mir-101-1,hsa-mir-101-2,hsa-mir-103-1,hsa-mir-103-2,hsa-mir-106a,hsa-mir-106b,hsa-mir-107,hsa-mir-10a,hsa-mir-10b,hsa-mir-1179,hsa-mir-1180,hsa-mir-1224,hsa-mir-1226,hsa-mir-1247,hsa-mir-1248,hsa-mir-1249,hsa-mir-1251,hsa-mir-1254,hsa-mir-125a,hsa-mir-125b-1,hsa-mir-125b-2,hsa-mir-126,hsa-mir-1266,hsa-mir-127,hsa-mir-1270-1,hsa-mir-1270-2,hsa-mir-1271,hsa-mir-1274b,hsa-mir-1277,hsa-mir-128-1,hsa-mir-128-2,hsa-mir-1287,hsa-mir-1288,hsa-mir-129-1,hsa-mir-129-2,hsa-mir-1291,hsa-mir-1292,hsa-mir-1296,hsa-mir-1301,hsa-mir-1306,hsa-mir-1307,hsa-mir-130a,hsa-mir-130b,hsa-mir-132,hsa-mir-133a-1,hsa-mir-133b,hsa-mir-134,hsa-mir-135a-1,hsa-mir-135a-2,hsa-mir-135b,hsa-mir-136,hsa-mir-138-1,hsa-mir-138-2,hsa-mir-139,hsa-mir-140,hsa-mir-141,hsa-mir-142,hsa-mir-143,hsa-mir-144,hsa-mir-145,hsa-mir-1468,hsa-mir-146a,hsa-mir-146b,hsa-mir-148a,hsa-mir-148b,hsa-mir-149,hsa-mir-150,hsa-mir-151,hsa-mir-152,hsa-mir-153-2,hsa-mir-155,hsa-mir-15a,hsa-mir-15b,hsa-mir-16-1,hsa-mir-16-2,hsa-mir-17,hsa-mir-181a-1,hsa-mir-181a-2,hsa-mir-181b-1,hsa-mir-181b-2,hsa-mir-181c,hsa-mir-181d,hsa-mir-182,hsa-mir-183,hsa-mir-185,hsa-mir-186,hsa-mir-187,...,hsa-mir-423,hsa-mir-424,hsa-mir-425,hsa-mir-429,hsa-mir-431,hsa-mir-4326,hsa-mir-450a-1,hsa-mir-450a-2,hsa-mir-450b,hsa-mir-451,hsa-mir-452,hsa-mir-454,hsa-mir-455,hsa-mir-484,hsa-mir-486,hsa-mir-491,hsa-mir-493,hsa-mir-495,hsa-mir-497,hsa-mir-499,hsa-mir-500a,hsa-mir-500b,hsa-mir-501,hsa-mir-502,hsa-mir-503,hsa-mir-505,hsa-mir-508,hsa-mir-509-1,hsa-mir-509-2,hsa-mir-509-3,hsa-mir-511-1,hsa-mir-511-2,hsa-mir-514-1,hsa-mir-514-2,hsa-mir-514-3,hsa-mir-532,hsa-mir-542,hsa-mir-548b,hsa-mir-548v,hsa-mir-550a-1,hsa-mir-550a-2,hsa-mir-551b,hsa-mir-574,hsa-mir-576,hsa-mir-577,hsa-mir-582,hsa-mir-584,hsa-mir-585,hsa-mir-589,hsa-mir-590,hsa-mir-592,hsa-mir-598,hsa-mir-616,hsa-mir-624,hsa-mir-625,hsa-mir-627,hsa-mir-628,hsa-mir-629,hsa-mir-639,hsa-mir-642a,hsa-mir-643,hsa-mir-651,hsa-mir-652,hsa-mir-653,hsa-mir-654,hsa-mir-659,hsa-mir-660,hsa-mir-664,hsa-mir-671,hsa-mir-675,hsa-mir-676,hsa-mir-7-1,hsa-mir-7-2,hsa-mir-708,hsa-mir-744,hsa-mir-758,hsa-mir-760,hsa-mir-766,hsa-mir-769,hsa-mir-874,hsa-mir-877,hsa-mir-887,hsa-mir-889,hsa-mir-891a,hsa-mir-9-1,hsa-mir-9-2,hsa-mir-92a-1,hsa-mir-92a-2,hsa-mir-92b,hsa-mir-93,hsa-mir-935,hsa-mir-937,hsa-mir-939,hsa-mir-940,hsa-mir-942,hsa-mir-95,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
12714.258,25457.58,12804.671,6117.175,4612.282,1629.2682,4214.221,44.36505,32399.949,1885.362,2606.982,6.425283,5573.474,22748.71,136.00182,21086.71,7.802129,3.365624,315.6038,78.02129,8609.879,40122.22,0.458949,25.39517,0.611932,0.917898,12.238634,0.764915,0.764915,115.04316,0.152983,2560.4753,3061.341,57.06263,5525.896,31.208518,452.37052,4.130539,3.365624,4.436505,3.365624,2.753693,67.46547,42.52925,33.04431,0.764915,0.458949,0.458949,0.611932,0.458949,7.649147,9.025993,23.100422,853.1858,101.42768,8.720027,135.23691,3.059659,0.305966,119.47967,732.7882,39.31661,417.79638,20.193747,5.048437,4.130539,126.97583,913.7670,2416.2124,885.1592,78296.20,89.95396,1343.4961,21.111644,37.02187,61178.486,16511.295,295.1041,22.182525,104.33436,4088.010,421.46797,8.873010,121.46845,197.5010,310.8613,1084.3430,9.637925,384.1401,4733.904,5539.359,1178.2745,28.30184,193.6764,44.21207,14914.000,5036.504,172.25878,307.8017,2.447727,...,240.1832,65.32371,91.17783,235.44073,2.753693,2.600710,3.059659,1.529829,6.884232,372.3605,83.37570,9.178976,65.62968,21.11164,70.83110,14.839344,6.731249,1.988778,13.003549,1.376846,250.7390,4.895454,30.29062,9.484942,1.070881,38.55170,23.71235,1.529829,1.070881,1.223863,8.720027,7.496164,7.649147,7.037215,6.425283,718.4078,118.8677,0.458949,0.917898,2.906676,1.682812,63.02897,118.40879,14.533378,1.835795,77.10340,64.09985,4.130539,65.32371,23.253405,3.518607,108.00595,5.201420,0.764915,135.69586,6.119317,25.85412,83.98763,1.22

---
### cortest

In [94]:
run <- readline("type? (u/m): ")

type? (u/m): m


In [107]:
if (run == "u"){
    corr_rna <- data.frame(mrna_p_uni, mirna_p_uni)
}else{
    corr_rna <- data.frame(mrna_p_multi, mirna_p_multi)
}

In [108]:
result_rcorr <- rcorr(as.matrix(corr_rna), type = "spearman")

result_rcorr$r <- result_rcorr$r[1:ncol(mrna_p_uni),(ncol(result_rcorr$r)-ncol(mirna_p_uni)+1):ncol(result_rcorr$r)]
result_rcorr$P <- result_rcorr$P[1:ncol(mrna_p_uni),(ncol(result_rcorr$P)-ncol(mirna_p_uni)+1):ncol(result_rcorr$P)]

In [109]:
head(result_rcorr$r)
head(result_rcorr$P)

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,hsa.let.7i,hsa.mir.1.2,hsa.mir.100,hsa.mir.101.1,hsa.mir.101.2,hsa.mir.103.1,hsa.mir.103.2,hsa.mir.106a,hsa.mir.106b,hsa.mir.107,hsa.mir.10a,hsa.mir.10b,hsa.mir.1179,hsa.mir.1180,hsa.mir.1224,hsa.mir.1226,hsa.mir.1247,hsa.mir.1248,hsa.mir.1249,hsa.mir.1251,hsa.mir.1254,hsa.mir.125a,hsa.mir.125b.1,hsa.mir.125b.2,hsa.mir.126,hsa.mir.1266,hsa.mir.127,hsa.mir.1270.1,hsa.mir.1270.2,hsa.mir.1271,hsa.mir.1274b,hsa.mir.1277,hsa.mir.128.1,hsa.mir.128.2,hsa.mir.1287,hsa.mir.1288,hsa.mir.129.1,hsa.mir.129.2,hsa.mir.1291,hsa.mir.1292,hsa.mir.1296,hsa.mir.1301,hsa.mir.1306,hsa.mir.1307,hsa.mir.130a,hsa.mir.130b,hsa.mir.132,hsa.mir.133a.1,hsa.mir.133b,hsa.mir.134,hsa.mir.135a.1,hsa.mir.135a.2,hsa.mir.135b,hsa.mir.136,hsa.mir.138.1,hsa.mir.138.2,hsa.mir.139,hsa.mir.140,hsa.mir.141,hsa.mir.142,hsa.mir.143,hsa.mir.144,hsa.mir.145,hsa.mir.1468,hsa.mir.146a,hsa.mir.146b,hsa.mir.148a,hsa.mir.148b,hsa.mir.149,hsa.mir.150,hsa.mir.151,hsa.mir.152,hsa.mir.153.2,hsa.mir.155,hsa.mir.15a,hsa.mir.15b,hsa.mir.16.1,hsa.mir.16.2,hsa.mir.17,hsa.mir.181a.1,hsa.mir.181a.2,hsa.mir.181b.1,hsa.mir.181b.2,hsa.mir.181c,hsa.mir.181d,hsa.mir.182,hsa.mir.183,hsa.mir.185,hsa.mir.186,hsa.mir.187,...,hsa.mir.423,hsa.mir.424,hsa.mir.425,hsa.mir.429,hsa.mir.431,hsa.mir.4326,hsa.mir.450a.1,hsa.mir.450a.2,hsa.mir.450b,hsa.mir.451,hsa.mir.452,hsa.mir.454,hsa.mir.455,hsa.mir.484,hsa.mir.486,hsa.mir.491,hsa.mir.493,hsa.mir.495,hsa.mir.497,hsa.mir.499,hsa.mir.500a,hsa.mir.500b,hsa.mir.501,hsa.mir.502,hsa.mir.503,hsa.mir.505,hsa.mir.508,hsa.mir.509.1,hsa.mir.509.2,hsa.mir.509.3,hsa.mir.511.1,hsa.mir.511.2,hsa.mir.514.1,hsa.mir.514.2,hsa.mir.514.3,hsa.mir.532,hsa.mir.542,hsa.mir.548b,hsa.mir.548v,hsa.mir.550a.1,hsa.mir.550a.2,hsa.mir.551b,hsa.mir.574,hsa.mir.576,hsa.mir.577,hsa.mir.582,hsa.mir.584,hsa.mir.585,hsa.mir.589,hsa.mir.590,hsa.mir.592,hsa.mir.598,hsa.mir.616,hsa.mir.624,hsa.mir.625,hsa.mir.627,hsa.mir.628,hsa.mir.629,hsa.mir.639,hsa.mir.642a,hsa.mir.643,hsa.mir.651,hsa.mir.652,hsa.mir.653,hsa.mir.654,hsa.mir.659,hsa.mir.660,hsa.mir.664,hsa.mir.671,hsa.mir.675,hsa.mir.676,hsa.mir.7.1,hsa.mir.7.2,hsa.mir.708,hsa.mir.744,hsa.mir.758,hsa.mir.760,hsa.mir.766,hsa.mir.769,hsa.mir.874,hsa.mir.877,hsa.mir.887,hsa.mir.889,hsa.mir.891a,hsa.mir.9.1,hsa.mir.9.2,hsa.mir.92a.1,hsa.mir.92a.2,hsa.mir.92b,hsa.mir.93,hsa.mir.935,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
ABAT,0.28561764,0.28468168,0.28541377,0.2562413,0.4409137,0.19125197,0.31112965,0.09214160,0.21399314,0.132805115,0.063135947,0.19486609,0.04323974,0.2227041,-0.009989806,0.18168844,0.05798350,0.13843944,1.918914e-01,0.40199240,-0.17258827,-0.18314336,0.34328759,0.46169030,0.2522948,-0.08348643,-0.06209948,-0.003665389,-0.26292890,0.26594384,-0.08016053,0.13442684,-0.08184598,0.43051617,0.1595033,0.07706422,-0.1827634,0.16103383,0.08717531,-0.133110926,0.03666945,-0.03039570,0.24209063,0.24874432,0.36409045,0.15378245,-0.08624009,-0.0203883,-0.13667261,0.04936092,0.05650079,0.1807432,0.05562969,-0.064396256,0.07292188,0.01294597,0.1141507,0.079010286,2.137556e-01,-0.14940228,0.22559540,0.3674729,-0.31188954,-0.21953531,0.186423872,0.15921601,0.1921694,0.15032898,-0.02095265,-0.39237327,0.3714577,0.1697989,0.1990177,0.21491057,-0.3232879,-0.07666574,-0.23227690,0.37081827,0.25235845,-0.32369567,0.38977852,0.24311927,0.07419146,-0.36130108,-0.22946900,0.093160967,-0.050412381,-0.17571124,-0.14831804,0.13388935,0.23144287,0.195931795,0.09254008,0.2729033,0.26791771,0.06254286,0.15414697,0.1329441,-0.10551385,-0.22629043,...,-0.01508665,0.08901863,-0.21971087,0.34838291,-0.1746239,-0.17813215,0.04341581,0.04603837,0.07241220,0.1757946,0.1742007,-0.007320916,-0.09724771,-0.003150774,0.12547493,0.02500232,-0.20536107,-0.17861131,0.06538782,0.039561112,-0.22639236,-0.1541377,-0.1912798,-0.06760263,-0.03249930,-0.139764619,-0.11964600,-0.09704869,-0.0

,hsa.let.7a.1,hsa.let.7a.2,hsa.let.7a.3,hsa.let.7b,hsa.let.7c,hsa.let.7d,hsa.let.7e,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,hsa.let.7i,hsa.mir.1.2,hsa.mir.100,hsa.mir.101.1,hsa.mir.101.2,hsa.mir.103.1,hsa.mir.103.2,hsa.mir.106a,hsa.mir.106b,hsa.mir.107,hsa.mir.10a,hsa.mir.10b,hsa.mir.1179,hsa.mir.1180,hsa.mir.1224,hsa.mir.1226,hsa.mir.1247,hsa.mir.1248,hsa.mir.1249,hsa.mir.1251,hsa.mir.1254,hsa.mir.125a,hsa.mir.125b.1,hsa.mir.125b.2,hsa.mir.126,hsa.mir.1266,hsa.mir.127,hsa.mir.1270.1,hsa.mir.1270.2,hsa.mir.1271,hsa.mir.1274b,hsa.mir.1277,hsa.mir.128.1,hsa.mir.128.2,hsa.mir.1287,hsa.mir.1288,hsa.mir.129.1,hsa.mir.129.2,hsa.mir.1291,hsa.mir.1292,hsa.mir.1296,hsa.mir.1301,hsa.mir.1306,hsa.mir.1307,hsa.mir.130a,hsa.mir.130b,hsa.mir.132,hsa.mir.133a.1,hsa.mir.133b,hsa.mir.134,hsa.mir.135a.1,hsa.mir.135a.2,hsa.mir.135b,hsa.mir.136,hsa.mir.138.1,hsa.mir.138.2,hsa.mir.139,hsa.mir.140,hsa.mir.141,hsa.mir.142,hsa.mir.143,hsa.mir.144,hsa.mir.145,hsa.mir.1468,hsa.mir.146a,hsa.mir.146b,hsa.mir.148a,hsa.mir.148b,hsa.mir.149,hsa.mir.150,hsa.mir.151,hsa.mir.152,hsa.mir.153.2,hsa.mir.155,hsa.mir.15a,hsa.mir.15b,hsa.mir.16.1,hsa.mir.16.2,hsa.mir.17,hsa.mir.181a.1,hsa.mir.181a.2,hsa.mir.181b.1,hsa.mir.181b.2,hsa.mir.181c,hsa.mir.181d,hsa.mir.182,hsa.mir.183,hsa.mir.185,hsa.mir.186,hsa.mir.187,...,hsa.mir.423,hsa.mir.424,hsa.mir.425,hsa.mir.429,hsa.mir.431,hsa.mir.4326,hsa.mir.450a.1,hsa.mir.450a.2,hsa.mir.450b,hsa.mir.451,hsa.mir.452,hsa.mir.454,hsa.mir.455,hsa.mir.484,hsa.mir.486,hsa.mir.491,hsa.mir.493,hsa.mir.495,hsa.mir.497,hsa.mir.499,hsa.mir.500a,hsa.mir.500b,hsa.mir.501,hsa.mir.502,hsa.mir.503,hsa.mir.505,hsa.mir.508,hsa.mir.509.1,hsa.mir.509.2,hsa.mir.509.3,hsa.mir.511.1,hsa.mir.511.2,hsa.mir.514.1,hsa.mir.514.2,hsa.mir.514.3,hsa.mir.532,hsa.mir.542,hsa.mir.548b,hsa.mir.548v,hsa.mir.550a.1,hsa.mir.550a.2,hsa.mir.551b,hsa.mir.574,hsa.mir.576,hsa.mir.577,hsa.mir.582,hsa.mir.584,hsa.mir.585,hsa.mir.589,hsa.mir.590,hsa.mir.592,hsa.mir.598,hsa.mir.616,hsa.mir.624,hsa.mir.625,hsa.mir.627,hsa.mir.628,hsa.mir.629,hsa.mir.639,hsa.mir.642a,hsa.mir.643,hsa.mir.651,hsa.mir.652,hsa.mir.653,hsa.mir.654,hsa.mir.659,hsa.mir.660,hsa.mir.664,hsa.mir.671,hsa.mir.675,hsa.mir.676,hsa.mir.7.1,hsa.mir.7.2,hsa.mir.708,hsa.mir.744,hsa.mir.758,hsa.mir.760,hsa.mir.766,hsa.mir.769,hsa.mir.874,hsa.mir.877,hsa.mir.887,hsa.mir.889,hsa.mir.891a,hsa.mir.9.1,hsa.mir.9.2,hsa.mir.92a.1,hsa.mir.92a.2,hsa.mir.92b,hsa.mir.93,hsa.mir.935,hsa.mir.937,hsa.mir.939,hsa.mir.940,hsa.mir.942,hsa.mir.95,hsa.mir.96,hsa.mir.98,hsa.mir.99a,hsa.mir.99b
ABAT,0.002607901,0.002697697,0.002627228,7.156903e-03,1.599305e-06,0.046355879,0.0009913532,0.34062805,0.02546029,0.1686213,0.5142654,0.04230414,0.6552791,0.0199330561,0.91788616,0.05865329,0.5492458,1.511135e-01,0.04561645,1.472431e-05,0.07271858,0.05662627,2.577708e-04,4.347283e-07,8.129905e-03,0.38809173,0.5212092,0.969825841,0.005741556,5.189254e-03,0.4073384,0.16343502,0.397515666,2.973816e-06,9.758234e-02,0.425750938,5.714999e-02,0.09436746,0.3673961,0.1676341,0.7050191,0.75370761,0.01120605,0.009103145,9.948318e-05,0.1103687,3.725777e-01,8.333337e-01,0.15645002,0.61025277,0.5595222,0.060001758,0.56560220,0.5058863857,4.511173e-01,0.8937131,2.372469e-01,0.41412315,0.02562768,0.121012229,1.834291e-02,8.468268e-05,0.0009619074,0.0218107220,0.05226622,0.09819527,4.529803e-02,1.186968e-01,0.828789907,2.445405e-05,6.988448e-05,7.753748e-02,3.801872e-02,0.0248224115,0.0006060958,4.281548e-01,0.01508121,7.208404e-05,0.008113328,0.0005959634,2.796532e-05,0.01085546,0.443254549,0.0001134636,0.01638488,0.3352901,0.6026566,0.06761318,0.123765311,0.1651406,0.015458694,0.04116739,0.3385350,0.0040910408,4.853804e-03,0.518232981,0.109516850,0.168172081,0.27485586,0.017977402,...,0.87626779,3.573135e-01,0.02170286,0.0002053907,6.935667e-02,0.06385886,0.653965500,0.6345265263,0.4542956737,0.0674809258,7.004502e-02,0.9397752,0.3144264,0.97406065,0.1935877720,0.796358447,0.0321772249,0.063136299,4.993442e-01,0.6829592,0.01792433,0.1095384,0.04632352,0.4848

In [116]:
for(i in 1:ncol(result_rcorr$r)-1){
    if(mean(result_rcorr$r[,i+1])>(-0.2)){
        print(1)
        result_rcorr$r <- result_rcorr$r[,-(i+1)]
        result_rcorr$P <- result_rcorr$P[,-(i+1)]
    }
}

In [117]:
head(result_rcorr$r)
head(result_rcorr$P)

,hsa.mir.129.1,hsa.mir.129.2,hsa.mir.142,hsa.mir.146a,hsa.mir.203,hsa.mir.21,hsa.mir.210,hsa.mir.22,hsa.mir.223,hsa.mir.33b,hsa.mir.34a,hsa.mir.3614,hsa.mir.362,hsa.mir.3934,hsa.mir.511.1,hsa.mir.511.2,hsa.mir.514.1,hsa.mir.616,hsa.mir.708,hsa.mir.766,hsa.mir.92b
ABAT,-0.08624009,-0.0203883,-0.39237327,-0.3232879,-0.2242517,-0.4630989,-0.2543972,-0.1338801,-0.35622278,-0.009526457,-0.2405986,-0.24947642,-0.3014920,-0.2998981,-0.3582522,-0.3436290,-0.1480400,-0.2597164,-0.4415902,-0.2978037,-0.19939765
ABCA8,-0.21841462,-0.2161346,-0.13673432,-0.2531091,-0.5062645,-0.3769901,-0.2576221,-0.1385784,-0.20512464,-0.264813270,-0.4888518,-0.24813270,-0.3182745,-0.4341303,-0.2701974,-0.2741266,-0.1930034,-0.3027523,-0.2693356,-0.1825966,-0.43767955
ABCB1,-0.30160849,-0.3676660,0.04817904,-0.0449912,-0.4326568,-0.2130016,-0.3200723,-0.1092392,-0.03077565,-0.274209990,-0.4213048,0.03872672,-0.2722269,-0.3443147,-0.2167454,-0.2344917,-0.2480122,-0.2370865,-0.2030303,-0.1450746,-0.37506255
ADAMTS9,-0.36923741,-0.4012974,-0.24993976,-0.3287647,-0.6401446,-0.4747567,-0.4125104,-0.4254842,-0.36401631,-0.359827634,-0.5649337,-0.32417756,-0.3959133,-0.5045779,-0.3981003,-0.4196182,-0.1880456,-0.3965341,-0.3077101,-0.3058104,-0.39982393
ADAMTSL2,-0.37983660,-0.3436356,-0.17798165,-0.2614123,-0.5342600,-0.3697248,-0.2339542,-0.1280512,-0.18198499,-0.153350014,-0.3792420,-0.28234640,-0.1956723,-0.4545362,-0.3299694,-0.3078213,-0.3393476,-0.2915763,-0.2822908,-0.2497452,-0.36068020
AGPAT5,-0.03299379,-0.1317270,-0.20977667,-0.2516727,-0.2483088,-0.2841905,-0.0931239,-0.1491706,-0.28672042,-0.116152349,-0.2652674,-0.15401724,-0.2835511,-0.2662960,-0.2488277,-0.2341025,-0.3307571,-0.1621536,-0.3767955,-0.1973218,-0.05739042


,hsa.mir.129.1,hsa.mir.129.2,hsa.mir.142,hsa.mir.146a,hsa.mir.203,hsa.mir.21,hsa.mir.210,hsa.mir.22,hsa.mir.223,hsa.mir.33b,hsa.mir.34a,hsa.mir.3614,hsa.mir.362,hsa.mir.3934,hsa.mir.511.1,hsa.mir.511.2,hsa.mir.514.1,hsa.mir.616,hsa.mir.708,hsa.mir.766,hsa.mir.92b
ABAT,3.725777e-01,8.333337e-01,2.445405e-05,0.0006060958,1.906776e-02,3.967304e-07,7.597943e-03,1.651701e-01,1.437101e-04,0.9216824018,1.173218e-02,0.0088944666,1.443281e-03,1.533926e-03,1.308214e-04,2.539082e-04,0.1244789334,6.386979e-03,1.534949e-06,0.001660883,3.764540e-02
ABCA8,2.251016e-02,2.399268e-02,1.562613e-01,0.0079200908,1.952683e-08,5.330348e-05,6.841666e-03,1.506995e-01,3.238048e-02,0.0053905557,6.917957e-08,0.0092807751,7.442800e-04,2.402633e-06,4.490557e-03,3.921113e-03,0.0443538344,1.375085e-03,4.624924e-03,0.057381168,1.943928e-06
ABCB1,1.436850e-03,8.390297e-05,6.188408e-01,0.6422595626,2.621678e-06,2.616538e-02,6.917151e-04,2.581689e-01,7.507291e-01,0.0039097602,5.065529e-06,0.6892983047,4.187819e-03,2.463100e-04,2.358752e-02,1.411738e-02,0.0093161201,1.305677e-02,3.422877e-02,0.132289074,5.861090e-05
ADAMTS9,7.780224e-05,1.528206e-05,8.764579e-03,0.0004823297,6.661338e-14,1.831560e-07,8.302449e-06,3.985801e-06,9.983302e-05,0.0001215659,1.564433e-10,0.0005841892,2.032657e-05,2.214071e-08,1.811353e-05,5.575016e-06,0.0502152996,1.967377e-05,1.134311e-03,0.001221618,1.653099e-05
ADAMTSL2,4.628113e-05,2.538334e-04,6.408720e-02,0.0060386593,2.189204e-09,7.599583e-05,1.434622e-02,1.845251e-01,5.823539e-02,0.1113858632,4.767243e-05,0.0029340071,4.144178e-02,6.874540e-07,4.584324e-04,1.129381e-03,0.0003064603,2.096686e-03,2.939854e-03,0.008818927,1.168145e-04
AGPAT5,7.334147e-01,1.721360e-01,2.857474e-02,0.0082934896,9.229329e-03,2.745923e-03,3.354832e-01,1.215965e-01,2.505565e-03,0.2290686928,5.308885e-03,0.1098194309,2.809871e-03,5.127936e-03,9.079156e-03,1.428277e-02,0.0004434013,9.206878e-02,5.381821e-05,0.039723103,5.533454e-01


In [118]:
if (run == "u"){
    write.csv(result_rcorr$r, "corr_uni.csv")
    write.csv(result_rcorr$P, "corr_uni_pval.csv")
}else{
    write.csv(result_rcorr$r, "corr_multi.csv")
    write.csv(result_rcorr$P, "corr_multi_pval.csv")
}

In [70]:
png(file="corr.png", res=300, width = 640, height = 2560)

corrplot(result_rcorr$r, method = "color", tl.cex = 0.3, tl.col = "black", cl.cex = 0.5)

dev.off()

png 
  2

In [91]:
mRNA_miRNA_corr <- data.frame(corr = colMeans(result_rcorr$r), pvalue = colMeans(result_rcorr$P))

In [92]:
mRNA_miRNA_corr

,corr,pvalue
hsa.mir.1266,-0.2098975,0.18639521
hsa.mir.129.1,-0.2162047,0.12011243
hsa.mir.129.2,-0.2190712,0.12493931
hsa.mir.146a,-0.2183204,0.12316655
hsa.mir.182,-0.2078968,0.10950278
hsa.mir.203,-0.3672360,0.03729054
hsa.mir.21,-0.2735295,0.08096982
hsa.mir.33b,-0.2269648,0.07540202
hsa.mir.34a,-0.3029407,0.05178899
hsa.mir.3614,-0.2039569,0.10693381
